<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Example-1" data-toc-modified-id="Example-1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Example 1</a></span><ul class="toc-item"><li><span><a href="#Document-Encoding" data-toc-modified-id="Document-Encoding-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Document Encoding</a></span></li><li><span><a href="#Padding" data-toc-modified-id="Padding-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Padding</a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Model</a></span><ul class="toc-item"><li><span><a href="#Not-Stacking" data-toc-modified-id="Not-Stacking-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Not Stacking</a></span></li><li><span><a href="#Stacking" data-toc-modified-id="Stacking-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Stacking</a></span></li></ul></li></ul></li></ul></div>

# LSTM

Example of using LSTM as the first layer without embedding layer. 

## Example 1

In [1]:
# from numpy import array
import numpy as np 
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM

print('done')

/Users/ijung/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


done


In [2]:
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = np.array([1,1,1,1,1,0,0,0,0,0])

### Document Encoding

In [3]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)

# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)

[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]


### Padding

In [4]:
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]


### Model

In [5]:
train = padded_docs.reshape(-1,4,1)

In [8]:
print(padded_docs.shape)
print(train.shape)

(10, 4)
(10, 4, 1)


#### Not Stacking

In [19]:
model = Sequential()
model.add(LSTM(64, input_shape=(4,1), return_sequences=False))

model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

print(model.summary())

history = model.fit(train, labels, 
#               validation_data=(x_val2, y_val), 
              epochs=10, 
              batch_size=512)

# evaluate the model
loss, accuracy = model.evaluate(train, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 16,961
Trainable params: 16,961
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
10/10 [==============================] - 1s 76ms/step - loss: 0.6845 - acc: 0.5000
Epoch 2/10
10/10 [==============================] - 0s 575us/step - loss: 0.6789 - acc: 0.5000
Epoch 3/10
10/10 [==============================] - 0s 557us/step - loss: 0.6737 - acc: 0.5000
Epoch 4/10
10/10 [==============================] - 0s 524us/step - loss: 0.6688 - acc: 0.6000
Epoch 5/10
10/10 [==============================] - 0s 492us/step - loss: 0.6642 - acc: 0.6000
Epoch 6/10
10/10 [==============================] - 0s 

#### Stacking

In [21]:
model = Sequential()
model.add(LSTM(64, input_shape=(4,1), return_sequences=True))
model.add(LSTM(32, input_shape=(4,1), return_sequences=False))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
print(model.summary())

history = model.fit(train, labels, 
              epochs=10, 
              batch_size=512)

# evaluate the model
loss, accuracy = model.evaluate(train, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 4, 64)             16896     
_________________________________________________________________
lstm_10 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 29,345
Trainable params: 29,345
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
10/10 [==============================] - 1s 138ms/step - loss: 0.7014 - acc: 0.5000
Epoch 2/10
10/10 [==============================] - 0s 848us/step - loss: 0.6955 - acc: 0.5000
Epoch 3/10
10/10 [==============================] - 0s 809us/step - loss: 0.6906 - acc: 0.5000
Epoch 4/10
10/10 [==============================] - 0s 816us/step - loss: 0.6865 - acc: 0.5000
Epoch 5/10
10/10 